In [1]:
import os
os.getcwd()
os.chdir("C:\\Users\somit\Downloads\project_ineuron\insurence_premium_prediction")
os.getcwd()

'C:\\Users\\somit\\Downloads\\project_ineuron\\insurence_premium_prediction'

In [2]:
from insurence_premium.config import ConfigurationManager

In [3]:
from pathlib import Path
from dataclasses import dataclass

import os

from insurence_premium.util.common import read_yaml
os.getcwd()
os.chdir("C:\\Users\somit\Downloads\project_ineuron\insurence_premium_prediction")
os.getcwd()

'C:\\Users\\somit\\Downloads\\project_ineuron\\insurence_premium_prediction'

In [4]:
read_yaml(CONFIG_FILE_PATH)

NameError: name 'CONFIG_FILE_PATH' is not defined

In [4]:
from insurence_premium import logger
from insurence_premium.constant import *
from insurence_premium.entity import DataIngestionConfig ,DataValidationConfig
from insurence_premium.util import read_yaml ,create_directories ,save_numpy_data ,save_object

ZipImportError: bad local file header: 'c:\\Users\\somit\\Downloads\\project_ineuron\\insurence_premium_prediction\\env\\lib\\site-packages\\insurencepremium-0.0.2-py3.8.egg'

In [3]:
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    transformed_train_file_path: Path
    transformed_test_file_path: Path
    preprocessing_dir: Path
    preprocessing_file_path: Path

In [4]:
class ConfigurationManager:
    def __init__(
        self ,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config =read_yaml(config_filepath)
        self.params =read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)->DataIngestionConfig:
        ingestion_config =self.config.data_ingestion
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(ingestion_config.root_dir),
            raw_data=Path(ingestion_config.raw_data),
            ingested_train_dir=Path(ingestion_config.ingested_train_dir),
            ingested_test_dir =Path(ingestion_config.ingested_test_dir)
        )
        return data_ingestion_config

    

    def get_data_validation_config(self)->DataValidationConfig:
        validation_config =self.config.data_validation

        data_validation_config =DataValidationConfig(
            root_dir=Path(validation_config.root_dir),
            schema_file_path=Path(validation_config.schema_file_path),
            report_file_path=Path(validation_config.report_file_path),
            report_page_file_path=Path(validation_config.report_page_file_path)
            
        )
        return data_validation_config

    def get_data_transformation_config(self)->DataTransformationConfig:
        try:
            transformation_config=self.config.data_transformation
            data_transformation_config =DataTransformationConfig(
                root_dir =Path(transformation_config.root_dir),
                transformed_train_file_path=Path(transformation_config.transformed_train_file_path),
                transformed_test_file_path=Path(transformation_config.transformed_test_file_path),
                preprocessing_dir=Path(transformation_config.preprocessing_dir),
                preprocessing_file_path=Path(transformation_config.preprocessing_file_path)
                )
            return data_transformation_config    
            logging.info(f"return: [{data_ingestion_config}]")
        except Exception as e:
            raise e     



In [5]:
con=ConfigurationManager()
con.get_data_transformation_config()

[2023-03-21 10:30:24,918: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-03-21 10:30:24,923: INFO: common]: yaml file: params.yaml loaded successfully
[2023-03-21 10:30:24,928: INFO: common]: created directory at: artifacts


DataTransformationConfig(root_dir=WindowsPath('artifacts/data_transformation'), transformed_train_file_path=WindowsPath('artifacts/data_transformation/train/insurence.npz'), transformed_test_file_path=WindowsPath('artifacts/data_transformation/test/insurence.npz'), preprocessing_dir=WindowsPath('artifacts/data_transformation/preprocessed_file'), preprocessing_file_path=WindowsPath('artifacts/data_transformation/preprocessed_file/preprocessing.pkl'))

In [5]:
from insurence_premium import logger

[2023-03-20 23:02:23,085: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-03-20 23:02:23,087: INFO: common]: yaml file: params.yaml loaded successfully
[2023-03-20 23:02:23,090: INFO: common]: created directory at: artifacts


DataIngestionConfig(root_dir=WindowsPath('artifacts/data_ingestion'), raw_data=WindowsPath('artifacts/data_ingestion/raw_data/insurence.csv'), ingested_train_dir=WindowsPath('artifacts/data_ingestion/train/insurence.csv'), ingested_test_dir=WindowsPath('artifacts/data_ingestion/test/insurence.csv'))

In [6]:
schema_file_path=ConfigurationManager().get_data_validation_config().schema_file_path
schema=read_yaml(schema_file_path)
schema[NUMERICAL_COLUMN_KEY][4]

[2023-03-21 07:42:27,953: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-03-21 07:42:28,063: INFO: common]: yaml file: params.yaml loaded successfully
[2023-03-21 07:42:28,067: INFO: common]: created directory at: artifacts
[2023-03-21 07:42:28,078: INFO: common]: yaml file: config\schema.yaml loaded successfully


'expenses'

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [7]:
class DataTransformation:
    def __init__(self):
        
        self.config=ConfigurationManager()
        self.data_validation_config=self.config.get_data_validation_config()
        self.data_ingestion_config=self.config.get_data_ingestion_config()
        self.data_transformation_config=self.config.get_data_transformation_config()
    @staticmethod
    def load_data(file_path:str ,schema_file_path:str)->pd.DataFrame:
        try:
            schema=read_yaml(schema_file_path)
            data_frame=pd.read_csv(file_path,index_col='Unnamed: 0')
            error_message =''

            for column in data_frame.columns:
                if column in list(schema.keys()):
                    data_frame[column].astype(schema[column])
                else:
                    error_message=f"{error_message} \ncolumn:[{column}] is not in the schema"    
            if len(error_message)>0:
                raise Exception(error_message)
            
            return data_frame      
        except Exception as e:
            raise e
    def get_data_transformer(self)->ColumnTransformer:
        try:
            schema_file_path=self.config.get_data_validation_config().schema_file_path
            dataset_schema=read_yaml(schema_file_path)
            numerical_columns=dataset_schema[NUMERICAL_COLUMN_KEY]
            categorical_columns=dataset_schema[CATEGORICAL_COLUMN_KEY]

            num_pipeline=Pipeline(steps=[
                ('imputer',SimpleImputer(strategy="median")),
                ('scaling',StandardScaler())
            ])
            
            cat_pipeline=Pipeline(steps=[
                ('imputer',SimpleImputer(strategy="most_frequent")),
                ('encoding',OneHotEncoder()),
                ('scaling',StandardScaler(with_mean=False))
            ])
            
            preprocessing =ColumnTransformer(
                [('num_pipeline',num_pipeline ,numerical_columns),
                ('cat_pipeline',cat_pipeline,categorical_columns)]
            )
            return preprocessing
            
        except Exception as e:
            raise e      
    def transformed_data(self):
        try:
            train_file_path=self.data_ingestion_config.ingested_train_dir
            test_file_path=self.data_ingestion_config.ingested_test_dir
            schema_file_path=self.data_validation_config.schema_file_path
            logger.info("loading train and test data")
            train_df=DataTransformation.load_data(file_path=train_file_path ,schema_file_path=schema_file_path)
            test_df=DataTransformation.load_data(file_path=test_file_path ,schema_file_path=schema_file_path)
            logger.info("loaded train and test data successfull")
            
            schema=read_yaml(schema_file_path)
            target_column=schema[TARGET_COLUMN_KEY]
            logger.info("extracting target feature")
            target_feature_train_df=train_df[target_column]
            input_feature_train_df=train_df.drop(columns=target_column , axis=1)
            logger.info("extracting target feature done sucessfully")
            
            logger.info("extracting target feature from test data")
            target_feature_test_df=test_df[target_column]
            input_feature_test_df=test_df.drop(columns=target_column ,axis=1)
            logger.info("extracting target feature from test data is done successfully")


            preprocessing_obj=self.get_data_transformer()
            logger.info("preprocessing obj generated successfully")
            input_feature_train_array=preprocessing_obj.fit_transform(input_feature_train_df)
            logger.info("train data transformed successfully")
            
            input_feature_test_array=preprocessing_obj.transform(input_feature_test_df)
            logger.info("test data transformed successfully")
            logger.info("train and test array concatination with target feature")
            train_array=np.c_[input_feature_train_array,np.array(target_feature_train_df)]
            test_array=np.c_[input_feature_test_array,np.array(target_feature_test_df)]
            logger.info("train and test array concatination with target feature is done successfully")

            transformed_train_file_path=self.data_transformation_config.transformed_train_file_path
            transformed_test_file_path=self.data_transformation_config.transformed_test_file_path

            train_dir ,file_name=os.path.split(transformed_train_file_path)
            os.makedirs(train_dir ,exist_ok=True)

            save_numpy_data(file_path=transformed_train_file_path,array=train_array)

            test_dir ,file_name=os.path.split(transformed_test_file_path)
            os.makedirs(test_dir ,exist_ok=True)

            save_numpy_data(file_path=transformed_test_file_path,array=test_array)
            preprocessing_obj_file_path=self.data_transformation_config.preprocessing_file_path

            save_object(file_path=preprocessing_obj_file_path ,obj=preprocessing_obj)
            logger.info("data transformation completed")




        except Exception as e:
            raise e

In [8]:
try:
       
    con_obj = ConfigurationManager()
    data_transformation= DataTransformation()
    data_transformation.get_data_transformer()
    data_transformation.transformed_data()
    
    
except Exception as e:
    raise e    

[2023-03-21 10:30:36,151: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-03-21 10:30:36,156: INFO: common]: yaml file: params.yaml loaded successfully
[2023-03-21 10:30:36,160: INFO: common]: created directory at: artifacts
[2023-03-21 10:30:36,166: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-03-21 10:30:36,169: INFO: common]: yaml file: params.yaml loaded successfully
[2023-03-21 10:30:36,171: INFO: common]: created directory at: artifacts
[2023-03-21 10:30:36,177: INFO: common]: yaml file: config\schema.yaml loaded successfully
[2023-03-21 10:30:36,179: INFO: 283229416]: loading train and test data
[2023-03-21 10:30:36,183: INFO: common]: yaml file: config\schema.yaml loaded successfully
[2023-03-21 10:30:36,204: INFO: common]: yaml file: config\schema.yaml loaded successfully
[2023-03-21 10:30:36,214: INFO: 283229416]: loaded train and test data successfull
[2023-03-21 10:30:36,219: INFO: common]: yaml file: config\schema.yaml loade

In [32]:
schema['target_column']

BoxList(['expenses'])

In [17]:
 def get_data_transformer()->ColumnTransformer:
        try:
            schema_file_path=ConfigurationManager().get_data_validation_config().schema_file_path
            dataset_schema=read_yaml(schema_file_path)
            numerical_columns=dataset_schema[NUMERICAL_COLUMN_KEY]
            categorical_columns=dataset_schema[CATEGORICAL_COLUMN_KEY]

            num_pipeline=Pipeline(steps=[
                ('imputer',SimpleImputer(strategy="median")),
                ('scaling',StandardScaler())
            ])
            
            cat_pipeline=Pipeline(steps=[
                ('imputer',SimpleImputer(strategy="most_frequent")),
                ('encoding',OneHotEncoder()),
                ('scaling',StandardScaler(with_mean=False))
            ])
            
            preprocessing =ColumnTransformer(
                [('num_pipeline',num_pipeline ,numerical_columns),
                ('cat_pipeline',cat_pipeline,categorical_columns)]
            )
            return preprocessing
            
        except Exception as e:
            raise e     

In [18]:
pre=get_data_transformer()
pre.fit_transform(df)

[2023-03-21 09:13:58,137: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-03-21 09:13:58,141: INFO: common]: yaml file: params.yaml loaded successfully
[2023-03-21 09:13:58,144: INFO: common]: created directory at: artifacts
[2023-03-21 09:13:58,149: INFO: common]: yaml file: config\schema.yaml loaded successfully


NameError: name 'df' is not defined

In [23]:
train_file_path=ConfigurationManager().get_data_ingestion_config().ingested_train_dir
test_file_path=ConfigurationManager().get_data_ingestion_config().ingested_test_dir
schema_file_path=ConfigurationManager().get_data_validation_config().schema_file_path
df=DataTransformation().load_data(file_path=train_file_path,schema_file_path=schema_file_path)




[2023-03-21 09:24:24,179: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-03-21 09:24:24,186: INFO: common]: yaml file: params.yaml loaded successfully
[2023-03-21 09:24:24,189: INFO: common]: created directory at: artifacts
[2023-03-21 09:24:24,196: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-03-21 09:24:24,200: INFO: common]: yaml file: params.yaml loaded successfully
[2023-03-21 09:24:24,202: INFO: common]: created directory at: artifacts
[2023-03-21 09:24:24,210: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-03-21 09:24:24,217: INFO: common]: yaml file: params.yaml loaded successfully
[2023-03-21 09:24:24,224: INFO: common]: created directory at: artifacts
[2023-03-21 09:24:24,236: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-03-21 09:24:24,244: INFO: common]: yaml file: params.yaml loaded successfully
[2023-03-21 09:24:24,248: INFO: common]: created directory at: artifacts
Unexpect

Traceback (most recent call last):
  File "box\box.py", line 503, in box.box.Box.__getitem__
KeyError: 'train_file_path'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "box\box.py", line 536, in box.box.Box.__getattr__
  File "box\box.py", line 524, in box.box.Box.__getitem__
box.exceptions.BoxKeyError: "'train_file_path'"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "box\box.py", line 538, in box.box.Box.__getattr__
AttributeError: 'ConfigBox' object has no attribute 'train_file_path'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "box\config_box.py", line 28, in box.config_box.ConfigBox.__getattr__
  File "box\box.py", line 552, in box.box.Box.__getattr__
box.exceptions.BoxKeyError: "'ConfigBox' object has no attribute 'train_file_path'"

During handling of the above exception, another ex

In [21]:
try:
    test_file_path=ConfigurationManager().get_data_ingestion_config().ingested_test_dir
    df_test=DataTransformation().load_data(file_path=test_file_path,schema_file_path=schema_file_path)
    df_test.head()
    target_column=schema[TARGET_COLUMN_KEY]

    df.drop(columns=target_column,axis=1)

    target_feature_test_df=df_test[target_column]
    input_feature_test_df=df_test.drop(columns=target_column , axis=1)
    pre.transform(input_feature_test_df)
except Exception as e:
    raise e    

[2023-03-21 09:16:20,786: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-03-21 09:16:20,793: INFO: common]: yaml file: params.yaml loaded successfully
[2023-03-21 09:16:20,795: INFO: common]: created directory at: artifacts
[2023-03-21 09:16:20,802: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-03-21 09:16:20,806: INFO: common]: yaml file: params.yaml loaded successfully
[2023-03-21 09:16:20,808: INFO: common]: created directory at: artifacts
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "box\box.py", line 503, in box.box.Box.__getitem__
KeyError: 'train_file_path'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "box\box.py", line 536, in box.box.Box.__getattr__
  File "box\box.py", line 524, in box.box.Box.__getitem__
box.exceptions.BoxKeyError: "'train_file_path'"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "box\box.py", line 538, in box.box.Box.__getattr__
AttributeError: 'ConfigBox' object has no attribute 'train_file_path'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "box\config_box.py", line 28, in box.config_box.ConfigBox.__getattr__
  File "box\box.py", line 552, in box.box.Box.__getattr__
box.exceptions.BoxKeyError: "'ConfigBox' object has no attribute 'train_file_path'"

During handling of the above exception, another ex

In [37]:
file_path=pd.read_csv(r"C:\Users\somit\Downloads\project_ineuron\insurence_premium_prediction\artifacts\data_ingestion\train\insurence.csv",index_col='Unnamed: 0')
file_path.head()

,age,sex,bmi,children,smoker,region,expenses
Unnamed: 0,,,,,,,
1215,55,male,38.3,0,no,southeast,10226.28
547,43,female,32.6,3,yes,southeast,40941.29
370,43,male,30.1,3,no,northwest,8410.05
525,46,female,35.5,0,yes,northeast,42111.66
1093,51,male,33.3,3,no,southeast,10560.49


In [38]:

file_path.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
expenses    float64
dtype: object

In [45]:
schema_file_path=Path("config\schema.yaml")

In [8]:
try:
    con_obj = ConfigurationManager()
    data_transformation_config = con_obj.get_data_transformation_config()
    data_transformation = DataTransformation()
    data_transformation.load_data(file_path=file_path ,schema_file_path=schema_file_path)
    
except Exception as e:
    raise e    

[2023-03-20 23:02:40,852: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-03-20 23:02:40,859: INFO: common]: yaml file: params.yaml loaded successfully
[2023-03-20 23:02:40,863: INFO: common]: created directory at: artifacts


NameError: name 'DataTransformation' is not defined

In [48]:
data_transformation.load_data(file_path=file_path ,schema_file_path=schema_file_path)
data_frame.columns

[2023-03-18 08:12:14,697: INFO: common]: yaml file: config\schema.yaml loaded successfully


NameError: name 'data_frame' is not defined

NameError: name 'SCHEMA_FILE_PATH' is not defined

In [31]:
import pandas as pd

In [98]:
df=pd.read_csv(r"C:\Users\somit\Downloads\project_ineuron\insurence_premium_prediction\artifacts\data_ingestion\train\insurence.csv",index_col='Unnamed: 0')

In [132]:
df.head()

,age,sex,bmi,children,smoker,region,expenses
Unnamed: 0,,,,,,,
1215,55,male,38.3,0,no,southeast,10226.28
547,43,female,32.6,3,yes,southeast,40941.29
370,43,male,30.1,3,no,northwest,8410.05
525,46,female,35.5,0,yes,northeast,42111.66
1093,51,male,33.3,3,no,southeast,10560.49


In [100]:
x,y =df.drop(columns=['expenses']) ,df[['expenses']]

In [101]:
x

,age,sex,bmi,children,smoker,region
Unnamed: 0,,,,,,
1215,55,male,38.3,0,no,southeast
547,43,female,32.6,3,yes,southeast
370,43,male,30.1,3,no,northwest
525,46,female,35.5,0,yes,northeast
1093,51,male,33.3,3,no,southeast
...,...,...,...,...,...,...
1050,29,male,33.3,2,no,northwest
487,31,male,39.5,1,no,southeast
648,46,female,28.9,2,no,southwest


In [102]:
x.isna().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
dtype: int64

In [103]:
x_cat=[i for i in x.columns if df[i].dtypes=='O']

In [104]:
x_cat

['sex', 'smoker', 'region']

In [105]:
x_num=[i for i in x.columns if df[i].dtypes!='O']

In [106]:
df_num=df[x_num]

In [107]:
import sklearn
from sklearn.impute import SimpleImputer

In [108]:
imp=SimpleImputer(strategy='median')

In [41]:
imp.fit_transform(df_num)

array([[55. , 38.3,  0. ],
       [43. , 32.6,  3. ],
       [43. , 30.1,  3. ],
       ...,
       [46. , 28.9,  2. ],
       [27. , 33.2,  2. ],
       [18. , 38.3,  0. ]])

In [42]:
imp.statistics_

array([40. , 30.2,  1. ])

In [44]:
class FeatureGenerater(TransformerMixin ,BaseEstimator):
    def __init__(self,strategy="median"):
        self.strategy=strategy
    def fit(self ,x):
        self.features=x.columns

        self.statistics_=[]
        for column in x.columns:
            self.statistics_.append(x[column].median)

          
        
    def transform(self ,x:pd.DataFrame):
        for idx,column in enumerate(x.columns):
            x[column].fillna(self.statistics_[idx]) 
            return x
        
   


In [109]:
class cstm_tra(BaseEstimator,TransformerMixin):
    def __init__(self ,strategy:"median"):
        self.strategy=strategy
       
    def fit(self ,x):
        self.mean_=[]
        self.std_=[]
        for column in x.columns:
            self.mean_.append(x[column].mean())
            self.std_.append(x[column].std())
        return self    
    def transform(self,x:pd.DataFrame):
        for idx ,column in enumerate(x.columns):
            x[column]=((x[column]-self.mean_[idx])/self.std_[idx])
            return x


In [110]:
for idx ,column in enumerate(df_num.columns):
    print(idx ,column)


0 age
1 bmi
2 children


In [111]:
a=cstm_tra(strategy="median")

In [112]:
a.mean_

AttributeError: 'cstm_tra' object has no attribute 'mean_'

In [113]:
a.fit(df_num)
a.transform(df_num)

C:\Users\somit\AppData\Local\Temp\ipykernel_4256\2736238233.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[column]=((x[column]-self.mean_[idx])/self.std_[idx])


,age,bmi,children
Unnamed: 0,,,
1215,1.122495,38.3,0
547,0.264982,32.6,3
370,0.264982,30.1,3
525,0.479360,35.5,0
1093,0.836658,33.3,3
...,...,...,...
1050,-0.735451,33.3,2
487,-0.592532,39.5,1
648,0.479360,28.9,2


In [88]:
df_num

,age,bmi,children
Unnamed: 0,,,
1215,1.122495,38.3,0
547,0.264982,32.6,3
370,0.264982,30.1,3
525,0.479360,35.5,0
1093,0.836658,33.3,3
...,...,...,...
1050,-0.735451,33.3,2
487,-0.592532,39.5,1
648,0.479360,28.9,2


In [20]:
num_pipeline=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy="median")),
    ('scaling',StandardScaler())
])

NameError: name 'SimpleImputer' is not defined

In [125]:
cat_pipeline=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy="most_frequent")),
    ('encoding',OneHotEncoder()),
    ('scaling',StandardScaler(with_mean=False))
])

In [126]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'expenses'], dtype='object')

In [127]:
preprocessing =ColumnTransformer(
    [('num_pipeline',num_pipeline ,['age', 'bmi', 'children']),
    ('cat_pipeline',cat_pipeline,['smoker', 'region','sex'])]
)

In [129]:
a=preprocessing.fit_transform(df)

In [130]:
pd.DataFrame(a)

,0,1,2,3,4,5,6,7,8,9,10
0,1.123021,1.237113,-0.917597,2.473503,0.000000,0.000000,0.000000,2.246681,0.000000,0.000000,2.001199
1,0.265106,0.317552,1.555288,0.000000,2.473503,0.000000,0.000000,2.246681,0.000000,2.001199,0.000000
2,0.265106,-0.085764,1.555288,2.473503,0.000000,0.000000,2.333916,0.000000,0.000000,0.000000,2.001199
3,0.479584,0.785399,-0.917597,0.000000,2.473503,2.333916,0.000000,0.000000,0.000000,2.001199,0.000000
4,0.837049,0.430481,1.555288,2.473503,0.000000,0.000000,0.000000,2.246681,0.000000,0.000000,2.001199
...,...,...,...,...,...,...,...,...,...,...,...
1064,-0.735795,0.430481,0.730993,2.473503,0.000000,0.000000,2.333916,0.000000,0.000000,0.000000,2.001199
1065,-0.592809,1.430705,-0.093302,2.473503,0.000000,0.000000,0.000000,2.246681,0.000000,0.000000,2.001199
1066,0.479584,-0.279356,0.730993,2.473503,0.000000,0.000000,0.000000,0.000000,2.330863,2.001199,0.000000
1067,-0.878781,0.414348,0.730993,2.473503,0.000000,0.000000,2.333916,0.000000,0.000000,0.000000,2.001199


In [134]:
test_df=pd.read_csv(r'C:\Users\somit\Downloads\project_ineuron\insurence_premium_prediction\artifacts\data_ingestion\test\insurence.csv' ,index_col='Unnamed: 0')

In [136]:
test_df.drop(columns="expenses" ,axis=1)

,age,sex,bmi,children,smoker,region
Unnamed: 0,,,,,,
269,19,male,27.8,0,no,northwest
897,30,male,23.0,2,yes,northwest
275,44,female,36.5,0,no,northeast
507,33,male,29.4,4,no,southwest
1073,35,female,28.0,0,yes,northwest
...,...,...,...,...,...,...
604,19,female,24.6,1,no,northwest
1018,27,female,31.3,1,no,northwest
1179,19,male,34.9,0,yes,southwest


In [138]:
b=preprocessing.transform(test_df)

In [139]:
pd.DataFrame(b

,0,1,2,3,4,5,6,7,8,9,10
0,-1.450725,-0.456815,-0.917597,2.473503,0.000000,0.000000,2.333916,0.0,0.000000,0.000000,2.001199
1,-0.664302,-1.231182,0.730993,0.000000,2.473503,0.000000,2.333916,0.0,0.000000,0.000000,2.001199
2,0.336599,0.946725,-0.917597,2.473503,0.000000,2.333916,0.000000,0.0,0.000000,2.001199,0.000000
3,-0.449824,-0.198693,2.379582,2.473503,0.000000,0.000000,0.000000,0.0,2.330863,0.000000,2.001199
4,-0.306838,-0.424550,-0.917597,0.000000,2.473503,0.000000,2.333916,0.0,0.000000,2.001199,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
263,-1.450725,-0.973060,-0.093302,2.473503,0.000000,0.000000,2.333916,0.0,0.000000,2.001199,0.000000
264,-0.878781,0.107828,-0.093302,2.473503,0.000000,0.000000,2.333916,0.0,0.000000,2.001199,0.000000
265,-1.450725,0.688603,-0.917597,0.000000,2.473503,0.000000,0.000000,0.0,2.330863,0.000000,2.001199
266,1.766457,1.398439,-0.917597,2.473503,0.000000,2.333916,0.000000,0.0,0.000000,2.001199,0.000000


In [51]:
ConfigurationManager().get_data_validation_config().schema_file_path

[2023-03-18 09:14:41,079: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-03-18 09:14:41,081: INFO: common]: yaml file: params.yaml loaded successfully
[2023-03-18 09:14:41,083: INFO: common]: created directory at: artifacts


WindowsPath('config/schema.yaml')

In [141]:
with open('preprocessing.pkl','wb') as pre_file:

    dill.dump(preprocessing ,pre_file)

In [19]:
CATEGORICAL_COLUMN_KEY

'categorical_columns'

In [ ]:
df_test=DataTransformation().load_data(file_path=test_file_path,schema_file_path=schema_file_path)
df_test.head()
target_column=schema[TARGET_COLUMN_KEY]

df.drop(columns=target_column,axis=1)

target_feature_train_df=df[target_column]
input_feature_train_df=df.drop(columns=target_column , axis=1)


In [6]:
from insurence_premium.config import ConfigurationManager

ZipImportError: bad local file header: 'c:\\Users\\somit\\Downloads\\project_ineuron\\insurence_premium_prediction\\env\\lib\\site-packages\\insurencepremium-0.0.2-py3.8.egg'

In [6]:
from insurence_premium.config import ConfigurationManager 